<a href="https://colab.research.google.com/github/rendalamili/ml-for-table-extraction/blob/main/Final_Metrics_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.9 MB/s eta 0:00:00


In [2]:
import os
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import math
from rapidfuzz import fuzz

In [3]:
# Upload zip file containing both ground truth and output csvs - metrics.zip
uploaded = files.upload()

Saving metrics.zip to metrics.zip


In [4]:
# Extract zip
with zipfile.ZipFile('metrics.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [6]:
# Ref: https://stackoverflow.com/questions/4623446/how-do-you-sort-files-numerically/4623518#4623518
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [7]:
def match_files(n):
    output_dir = '/content/output_csvs/'
    ground_truth_dir = '/content/ground_truth_csvs/'
    # Get sorted list of files
    output_files = os.listdir(output_dir)
    sort_nicely(output_files)

    ground_truth_files = os.listdir(ground_truth_dir)
    sort_nicely(ground_truth_files)

    # match files based on first 10 characters
    matched_files = []
    for output_file in output_files:
      output_name = output_file[:10]
      ground_truth_file = None
      for ground_truth_file in ground_truth_files:
        if ground_truth_file[:10] == output_name:
          matched_files.append((output_file, ground_truth_file))
    return(matched_files[n-1])

In [8]:
def format_output_df(output_df):
    ## format output_csv df for accuracy comparison
    # header should be first row, not row[0]
    headers = pd.DataFrame([output_df.columns])
    output_df.columns = range(len(output_df.columns))
    output_df = pd.concat([headers, output_df])
    # output_df = output_df.astype('object')
    output_df = output_df.reset_index(drop=True)
    return output_df

In [31]:
def load_output_dfs(file):
    output_dir = '/content/output_csvs/'
    output_csv = pd.read_csv(output_dir + file)
    output_df = pd.DataFrame(output_csv)
    output_df = format_output_df(output_df)
    return output_df
df = load_output_dfs(match_files(1)[0])
df

,0,1,2,3,4,5,6
0,Form,Average,retail price,Preparation yield factor,Size ec,of a cup equivalent,Average price per cup equivalent
1,Fresh1,$1.85,per pound,0.9,0.243,Pounds,$0.50
2,Applesauce?,$1.17,per pound,1.0,0.54,Pounds,$0.63
3,Juice,NaN,NaN,NaN,NaN,NaN,NaN
4,Ready to drink,$0.87,per pint,1.0,8.0,Fluid ounces,$0.43
5,Frozen 4,$0.61,per pint,1.0,8.0,Fluid ounces,$0.30


In [32]:
def format_ground_truth_df(ground_truth_df):
    ## format ground_truth_csv df for accuracy comparison
    # first row should be header, not row[0]
    # ground_truth_df.columns = ground_truth_df.iloc[0]
    # ground_truth_df = ground_truth_df[1:]
    # ground_truth_df

    # Continuously remove rows where all values from column 2 onward are NaN
    while not ground_truth_df.empty:
        if ground_truth_df.iloc[-1, 1:].isna().all():
            # Drop the last row
            ground_truth_df = ground_truth_df.iloc[:-1]
        else:
            break  # Stop when a valid row is found
    ground_truth_df.columns = range(len(ground_truth_df.columns))
    return ground_truth_df


In [33]:
def load_ground_truth_dfs(file):
    # Read ground_truth_csv 1 as df
    ground_truth_dir = '/content/ground_truth_csvs/'
    ground_truth_csv =pd.read_csv(ground_truth_dir + file, encoding='unicode_escape')
    ground_truth_df = pd.DataFrame(ground_truth_csv)
    ground_truth_df = format_ground_truth_df(ground_truth_df)
    return ground_truth_df
df2 = load_ground_truth_dfs(match_files(1)[1])
df2


,0,1,2,3,4,5,6
0,Form,Average retail price,NaN,Preparation yield factor,Size of a cup equivalent,NaN,Average price per cup equivalent
1,Fresh1,$1.85,per pound,0.9,0.243,Pounds,$0.50
2,Applesauce2,$1.17,per pound,1,0.540,Pounds,$0.63
3,Juice,NaN,NaN,NaN,NaN,NaN,NaN
4,Ready to drink3,$0.87,per pint,1,8,Fluid ounces,$0.43
5,Frozen4,$0.61,per pint,1,8,Fluid ounces,$0.30


In [34]:
# fixing y_pred and y_true
def is_nan(value):
    return isinstance(value, float) and math.isnan(value)

def fix_lowercase(lst):
  #creates copy of list
  new_list = lst[:]
  for i in range(len(lst)):
    if is_nan(lst[i]):
      continue
    if isinstance(lst[i], str) and lst[i][0].islower():
      if i > 0 and not is_nan(lst[i-1]) and isinstance(new_list[i-1], str):
        if all(part.isalpha() for part in new_list[i-1].split()):
          new_list[i-1] += " " + lst[i]
      new_list[i] = np.nan
  return new_list

def normalise_values(pred, true):
  normalised_pred = []
  for x in pred:
    if isinstance(x, float):
      x = f"{x:.3f}".rstrip('0').rstrip('.')
    normalised_pred.append(str(x))

    normalised_true = []
  for x in true:
    if isinstance(x, str) and '.' in x and not x.startswith('$'):
      x = x.rstrip('0').rstrip('.')
    normalised_true.append(str(x))

  return normalised_pred, normalised_true

In [35]:
def structure_accuracy(output_df, ground_truth_df):
    # Structure metrics - no. of rows and columns match
    structure_metrics = {
        "row_count_match": output_df.shape[0] == ground_truth_df.shape[0],
        "column_count_match": output_df.shape[1] == ground_truth_df.shape[1]}
    structure_accuracy = sum(structure_metrics.values()) / len(structure_metrics)
    return structure_accuracy

In [36]:
structure_accuracy(df, df2)

1.0

In [38]:
# Row Accuracy using rapidfuzz
def row_accuracy(output_df, ground_truth_df, match_threshold=0.8, cell_match_threshold=80):
    ot_row_number = len(output_df)
    gt_row_number = len(ground_truth_df)
    min_row = min(ot_row_number, gt_row_number)

    matching_rows = 0

    for i in range(min_row):
        # Get the rows from both dataframes
        true_row = ground_truth_df.iloc[i]
        pred_row = output_df.iloc[i]

        # Count matching cells using rapidfuzz
        total_cells = len(true_row)
        matching_cells = sum(
            1 for true_cell, pred_cell in zip(true_row, pred_row)
            if fuzz.ratio(str(true_cell), str(pred_cell)) >= cell_match_threshold
        )

        # Check if matching cells meets the match threshold
        if matching_cells / total_cells >= match_threshold:
            matching_rows += 1

    # Calculate row accuracy
    row_accuracy = matching_rows / gt_row_number
    return row_accuracy

In [39]:
row_accuracy(df, df2)

0.5

In [42]:
def cell_accuracy(output_df, ground_truth_df):
    y_true = ground_truth_df.values.flatten()
    y_pred = output_df.values.flatten()
    y_pred = fix_lowercase(y_pred)
    y_true = fix_lowercase(y_true)
    y_pred, y_true = normalise_values(y_pred, y_true)
    if output_df.shape[0] == ground_truth_df.shape[0] and output_df.shape[1] == ground_truth_df.shape[1]:
        accuracy = accuracy_score(y_true, y_pred)
    else:
        accuracy = 0
    return accuracy

In [43]:
cell_accuracy(df, df2)

0.7857142857142857

In [56]:
def cell_acc(output_df, ground_truth_df, match_threshold=0.8, cell_match_threshold=80): # Partial Cell Accuracy
    y_true = ground_truth_df.values.flatten()
    y_pred = output_df.values.flatten()
    y_pred = fix_lowercase(y_pred)
    y_true = fix_lowercase(y_true)
    y_pred, y_true = normalise_values(y_pred, y_true)
    matching_cells = 0
    total_cells = len(y_true)
    matching_cells = sum(
        1 for true_cell, pred_cell in zip(y_true, y_pred)
        if fuzz.ratio(str(true_cell), str(pred_cell)) >= cell_match_threshold
    )
    if matching_cells / total_cells >= match_threshold:
        matching_cells += 1
    cell_accuracy = matching_cells / total_cells
    return cell_accuracy


In [57]:
cell_acc(df, df2)

0.9285714285714286

In [60]:
# create df of all results and all tables
def results_df():
    results = []
    for n in range(1,46):
        output_file = match_files(n)[0]
        ground_truth_file = match_files(n)[1]
        output_df = load_output_dfs(output_file)
        ground_truth_df = load_ground_truth_dfs(ground_truth_file)
        results.append({
            'Output File': output_file,
            'Ground Truth File': ground_truth_file,
            'Structure Accuracy': structure_accuracy(output_df, ground_truth_df),
            'Row Accuracy': row_accuracy(output_df, ground_truth_df),
            'Cell Accuracy': cell_accuracy(output_df, ground_truth_df),
            'Cell Accuracy (Partial)': cell_acc(output_df, ground_truth_df)
        })
    # pd.options.display.float_format = '{:.2%}'.format
    return pd.DataFrame(results)

results_df()

,Output File,Ground Truth File,Structure Accuracy,Row Accuracy,Cell Accuracy,Cell Accuracy (Partial)
0,table_1_Apples-2022_page_1_table_1.csv,table_1_Apples-2022.csv,1.0,0.500000,0.785714,0.928571
1,table_2_Apricots-2022_page_1_table_1.csv,table_2_Apricots-2022.csv,1.0,0.666667,0.738095,0.857143
2,table_3_Artichoke-2022_page_1_table_1.csv,table_3_Artichoke-2022.csv,1.0,0.666667,0.761905,0.904762
3,table_4_Beets-2022_page_1_table_1.csv,table_4_Beets-2022.csv,1.0,0.500000,0.714286,0.928571
4,table_5_Black-beans-2022_page_1_table_1.csv,table_5_Black-beans-2022.csv,1.0,0.666667,0.666667,0.857143
5,table_6_Blackberries-2022_page_1_table_1.csv,table_6_Blackberries-2022.csv,1.0,0.666667,0.714286,0.904762
6,table_7_Blueberries-2022_page_1_table_1.csv,table_7_Blueberries-2022.csv,1.0,0.666667,0.714286,0.904762
7,table_8_Cabbage-2022_page_1_table_1.csv,table_8_Cabbage-2022.csv,1.0,0.750000,0.714286,0.892857
8,table_9_Cantaloupe-2022_page_1_table_1.csv,table_9_Cantaloupe-2022.csv,0.0,0.000000,0.000000,0.000000
9,table_10_Carrots-2022_page_1_table_1.csv,table_10_Carrots-2022.csv,1.0,0.857143,0.755102,0.897959


In [61]:
#Summary of results
result_df = results_df()
result_df.describe()

,Structure Accuracy,Row Accuracy,Cell Accuracy,Cell Accuracy (Partial)
count,45.000000,45.000000,45.000000,45.000000
mean,0.966667,0.597690,0.691965,0.854928
std,0.165145,0.153562,0.154099,0.158291
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.500000,0.714286,0.857143
50%,1.000000,0.666667,0.714286,0.904762
75%,1.000000,0.666667,0.750000,0.928571
max,1.000000,0.857143,0.800000,0.942857


In [69]:
#Export df as csv
result_df = results_df()
result_df.to_csv('results.csv', index=False)

# Testing Functions

In [65]:
def test(n):
    output_file = match_files(n)[0]
    ground_truth_file = match_files(n)[1]
    output_df = load_output_dfs(output_file)
    ground_truth_df = load_ground_truth_dfs(ground_truth_file)
    results = cell_accuracy(output_df, ground_truth_df)
    print(results)

In [66]:
test(9)

0


In [67]:
def test_all():
    for n in range(1,46):
        test(n)
        print("\n" + "-"*50 + "\n")

In [68]:
test_all()

0.7857142857142857

--------------------------------------------------

0.7380952380952381

--------------------------------------------------

0.7619047619047619

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.6666666666666666

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0

--------------------------------------------------

0.7551020408163265

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.75

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.7142857142857143

--------------------------------------------------

0.7142857142857143

---

In [ ]:
def detailed_results(n):
    output_file = match_files(n)[0]
    ground_truth_file = match_files(n)[1]
    output_df = load_output_dfs(output_file)
    ground_truth_df = load_ground_truth_dfs(ground_truth_file)
    print("OUTPUT DF", n, output_df)
    print("GROUND TRUTH DF", n, ground_truth_df)
    print(f"STRUCTURE ACCURACY FOR FILE {n}: {structure_accuracy(output_df, ground_truth_df):.2%}")
    print(f"ROW ACCURACY FOR FILE {n}: {row_accuracy(output_df, ground_truth_df):.2%}")
    print(f"CELL ACCURACY FOR FILE {n}: {cell_accuracy(output_df, ground_truth_df):.2%}")
    print(f"CELL ACCURACY (PARTIAL) FOR FILE {n}: {cell_acc(output_df, ground_truth_df):.2%}")

In [ ]:
def all_detailed_results():
    for n in range(1,46):
        detailed_results(n)
        print("\n" + "-"*50 + "\n")